In [ ]:
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

# importing libraries/modules, and functions
import pandas as pd
import numpy as np
import scipy.stats as stats

# visualization libraries
import matplotlib as mlp
mlp.rcParams['figure.dpi'] = 300
from matplotlib import pyplot as plt
import seaborn as sns
sns.set()

# created zillow library and functions
from acquire import get_zillow_dataset, clean_zillow_dataset, zillow_outliers, train_validate_test_split, select_kbest, recursive_feature_eng
from prepare import display_all

# sklearn library for data science
from sklearn.metrics import mean_squared_error
from sklearn.feature_selection import SelectKBest, RFE, f_regression
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression, LassoLars, TweedieRegressor
from sklearn.preprocessing import RobustScaler, StandardScaler, MinMaxScaler

----
### ``Data Acquisition & Preparation:``

In [ ]:
# initial query for MySQL zillow data
# query = '''
# SELECT *
#     FROM properties_2017
#         JOIN predictions_2017 USING (id)
#             JOIN propertylandusetype USING (propertylandusetypeid)
#                 WHERE transactiondate = 2017
#                     AND propertylandusedesc = "Single Family Residential"'''

In [ ]:
# pulling the MySQL zillow data
# url = get_connection(user, password, host, "zillow")
# df = pd.read_sql(query, url)

In [ ]:
# creating a csv file for storing
# df.to_csv("/Users/mijailmariano/codeup-data-science/regression-exercises/zillow_regression.csv")

In [ ]:
# can now pull the zillow data using the following function:
zillow_df = get_zillow_dataset()
display_all(zillow_df.head()) # checks out!

In [ ]:
# exploring the dataset further
initial_shape = zillow_df.shape

In [ ]:
zillow_df.info()

In [ ]:
# creating a function that bypasses pd row/cols limits
# with pd.option_context('display.max_rows', None):
#     display(zillow_df.isnull().sum())

display_all(zillow_df.isnull().sum())

In [ ]:
# checking total null percentage for ea. column
display_all(round(zillow_df.isnull().mean(), 3))

In [ ]:
# cleaning df for for feature with NULL % > 20% 
# though this may not be a necessary step at this time
# consider first, choosing features/variables from research/domain knowledge that may be closely tied to home value
# from these features/variables turn them into questions/hypothesis for exploration

dropped_cols = []
for col in zillow_df.columns:
    if zillow_df[col].isnull().mean() > 0.2:
        dropped_cols.append(col)
        zillow_df = zillow_df.drop(columns = col)

# returning initial shape vs. null drop shape
print(f'initial df shape: {initial_shape}')
print(f'shape after null drop: {zillow_df.shape}')
print('dropped columns:', *dropped_cols, sep = '\n- ')

In [ ]:
# let's see all "id" columns that remain
# i presume that i will not need most, if not all of them - but let's check anyways

mask = zillow_df.columns.str.contains("id")
zillow_df.iloc[:, mask].columns

In [ ]:
# will drop column "parcelid.1" from the mysql predictions_2017 table as this does not appear to have any significance to my current zillow dataframe
zillow_df[["parcelid", "parcelid.1"]] 

In [ ]:
zillow_df = zillow_df.drop(columns = "parcelid.1")

print(zillow_df.shape)
zillow_df.head()

In [ ]:
# remaining columns/features include:
pd.Series(zillow_df.columns.sort_values())

----
### <u>**``Initial Questions for Exploration:``**</u>

**``Target Variable:``**
* Tax_Value_Dollar_Count

``Features That May* Lead to Data Leakage:``
- land tax value dollar count
- structure tax value dollar count
- tax amount
- land tax value dollar count
- structure tax value dollar count


**1. Does the home location matter? (must try) model per county**

I presume location matters in determining the final value/price of a home, as there could be economic factors (e.g., cost-of-living/inflationary considerations). Additionally, homes typically closer to metropolitan or labor creating areas (employment) may be more costly to live-in than more rural/less-labor promoted areas.

* Homes closer to schools (positive)
* Home closer to parks/recreational areas (positive)
* Homes near hospitals/hospice care (positive)
* Areas with “less” crime rates or law-enforcement? (positive)
* Areas near bodies of water may be less attractive? Given the additional insurance costs/natural disaster considerations (negative)

``Features:``
1. Fips
2. Latitude 
3. Longitude 
4. Parcel_id
5. Region_City_id
6. Region_id_County
7. Region_id_Zip


**2. Does the size of the home matter?**

I presume that the larger the home (as measured by sq. ft.), the higher the home value will be. A larger home can be more attractive to home buyers since the space can act as both an initial family home, but also a “forever home”. Meaning that people who might not initially have a use for the additional space - may see the potential benefits of having it when they are ready to either 1. Expand their family/or use of the space or 2. View the space as a future investment for someone willing to pay the same or more for this space (investment thinking). 

* Consider the total number of baths 
* Consider the total number of rooms

``Features:``
1. Bathroom_Count
2. Bedroom_Count
3. Calculated_Finished_Sq_Feet
4. Full_Bath_Count

**3. Does the period of when a home is purchased matter?**

I presume that when* a home is placed on the market/is purchased matters. Home buyers may be more reluctant to purchase a home in the colder regional months (e.g., typically winter) when the weather may be less favorable for moving. There may also be *renter factors or periods in the year when leases end and renters make the decision to purchase a home, subsequently driving more buyers to the market and thus potentially increasing home values. 

*(higher demand + “same” or not enough supply = more competition/higher home purchase price)*

* Consider seasonal patterns (e.g., summer months vs. winter) 

``Features:``
1. Assessment_Year
2. Year_built
3. Transaction_Date


**<u>Features not taken forward after initial query pull:</u>**

- **parcel_id** 
  - Though may be needed in final predictions/report
<br> </br>

- **room_count** 
  - This is the total number of rooms in a home. Omitting this since it may be too closely associated to other features such as bathrooms and bedrooms. Additionally, this analysis presumes that all homes have standard rooms such as kitchens and living-rooms.
<br> </br>
- calculated bath and bedroom
- full bath count
- census tract and block
- finished squared feet 12
- property county land use code
- raw census tract and block
- assessment tax year
- land tax value dollar count
- structure tax value dollar count
- tax amount
- structure tax value dollar count
- assessment tax year



-----

In [ ]:
exploration_cols = [
'taxvaluedollarcnt',
'bathroomcnt',
'bedroomcnt',
'calculatedfinishedsquarefeet',
'fips',
'latitude',
'longitude',
'lotsizesquarefeet',
'regionidcity',
'regionidcounty',
'regionidzip',
'yearbuilt',
'transactiondate'
]

In [ ]:
# creating new dataframe for exploration:

df = zillow_df[exploration_cols]
df.info()

In [ ]:
# let's rename columns for clarity

df = df.rename(columns = {
'taxvaluedollarcnt': "home_value",
'bathroomcnt': "bathroom_count",
'bedroomcnt': "bedroom_count",
'calculatedfinishedsquarefeet': "living_sq_feet",
'fips': "fips_code",
'lotsizesquarefeet': "property_sq_feet",
'parcelid': "property_id",
'regionidcity': "city_id",
'regionidcounty': "county_id",
'regionidzip': "zip_code",
'transactiondate': "purchase_date",
'yearbuilt': "year_built",
})
df.columns.to_list() # checks out!

In [ ]:
# let's confirm the df shape:
initial_shape = df.shape
initial_shape

In [ ]:
# let's recheck the number of null values in columns/features:
df.isnull().sum()

In [ ]:
# let's check the null percentage per feature/column:
df.isnull().mean()

In [ ]:
(df.isnull().mean() * 100).round(3)

**<u>``Notes on remaining null values:``</u>**

1. since it is not know how this data was collected, it is therfore difficult to properly determine the cause of missing information (MCAR/MAR/MNAR)
2. area specific information/values would be relatively difficult here to predict and given the relatively low percentage of null values - i dont believe at this current stage it is beneficial to handle these values in a meaningful way that would generate any additional insight into this analysis
3. most features at this stage are comprised of <2% of null values - therefore, i will drop these and note which features contained nulls:


|feature| null % |
|----|----|
|'home_value'|0.014|
|'living_sq_feet'|0.41|
|'property_sq_feet'|0.62|
|'city_id'|1.88|
|'zip_code'|0.18|
|'year_built'|0.44|

------

In [ ]:
# dropping nulls and rechecking df shape:
df = df.dropna()
shape_after_nulls = df.shape
shape_after_nulls

In [ ]:
df.info()

In [ ]:
# let's check the total number of unique values/feature options per column:

for col in df.columns:
    print(f'Column/Feature:  {col}'),
    print(f'Feature/value Type:  {df[col].dtype}'),
    print(f'Feature min:  {df[col].min()} | Feature max:  {df[col].max()}'),
    print(df[col].unique()),
    print("-----------------------------------------------"),
    print()

In [ ]:
col_lst = df.columns.to_list()
col_lst

In [ ]:
# converting the following feature to "int" type
# add to function::: 

to_interger = ["bedroom_count", "city_id", "county_id", "zip_code"]
df[to_interger] = df[to_interger].astype("int")

df.info()

In [ ]:
# summary stats:

summary_stats = df.describe().T
summary_stats["range"] = summary_stats["max"] - summary_stats["min"]
summary_stats.round()

----
### <u>``Univariate Feature Analysis:``</u>

In [ ]:
# beginning with the target variable:

plt.figure(figsize = (12, 8))
sns.set(font_scale = .8)
ax = sns.histplot(df["home_value"])

# removing axes scientific notation 
ax.ticklabel_format(style = "plain")

plt.axvline(df["home_value"].mean(), linewidth = 1, color = 'purple', alpha = 0.4, label = "mean")
plt.legend()
plt.title("Histogram: Home Values")
plt.show()

In [ ]:
# box plot where outliers are present
plt.figure(figsize = (10, 6))
df.boxplot(column = "home_value", color = "black")

plt.semilogy()
plt.title("Home Value: Outliers Identified")
plt.show()

In [ ]:
# mansion style homes 
# where living sq footage > 8K sq ft.
# 90th percent quartile for homes under this criteria = ~900K

over_8k = (df[df["living_sq_feet"] <= 8_000].home_value)
np.quantile(over_8k, q = [.90])

In [ ]:
# percentage of homes in the dataset that are >= 1.5MIL 
# cleaning dataset for values over 1.5mil as these homes may be considered to be 

print(f'Total Num. of homes that are equal to or greater than 900K in value: {(df["home_value"] >= 900_000).sum()}')
print("---------")
print(f'Total Num. of homes that have 8K or greater living sq. footage: {(df["living_sq_feet"] >= 8_000).sum()}')
print("---------")
print(f'Total Num. of homes that have eight (8) or greater bedrooms: {(df["bedroom_count"] >= 8).sum()}')
print(f'Total Num. of homes that have 6 or greater bedrooms: {(df["bedroom_count"] >= 6).sum()}')
print(f'Total Num. of homes that have 0 bedrooms: {(df["bedroom_count"] == 0).sum()}')
print("---------")
print(f'Total Num. of homes that have 6.5 or greater baths: {(df["bathroom_count"] >= 6.5).sum()}')
print(f'Total Num. of homes that have eight (8) or greater baths: {(df["bathroom_count"] >= 8).sum()}')
print(f'Total Num. of homes that have 0 baths: {(df["bathroom_count"] == 0).sum()}')
print("----------------------------------")
print(f'Percentage of homes that are equal to or greater than 900K in value: {(df["home_value"] >= 900_000).mean().round(3)}')
print(f'Percentage of homes that have 8K or greater living sq. footage: {(df["living_sq_feet"] >= 8_000).mean().round(3)}')
print(f'Percentage of homes that have greater than 6 bedrooms: {(df["bedroom_count"] > 6).mean().round(3)}')
print(f'Percentage of homes that have greater than 6.5 bathrooms: {(df["bathroom_count"] > 6.5).mean().round(3)}')

In [ ]:
# dropping identified outlier cutoffs in the dataset 
# as these homes may represent houses larger than the majority homes, therefore making the analysis or future prediction less accurate when comparing these outliers.

df = df[df["home_value"] <= 900_000]
df = df[df["living_sq_feet"] <= 8_000]

# also removing homes with 0 bedrooms as this appears to be slightly "irregular"
df = df[(df["bedroom_count"] > 0) & (df["bedroom_count"] <= 6)]

# also removing homes with 0 bathrooms as this appears to be slightly "irregular"
df = df[(df["bathroom_count"] > 0) & (df["bathroom_count"] <= 6.5)]

shape_after_outliers = df.shape
shape_after_outliers

In [ ]:
# let's check target again
from acquire import plot_target, plot_continuous, plot_discrete

plot_target(df["home_value"])

In [ ]:
df.info()

In [ ]:
# setting and plotting continuous features/variables

continuous_vars = [
 'living_sq_feet',
 'latitude',
 'longitude',
 'property_sq_feet',
 'city_id',
 'county_id',
 'zip_code',
 'year_built'
]

plot_continuous(df, continuous_vars)

In [ ]:
# plotting categorical/potential discrete features/columns:

discrete_vars = [
    'bathroom_count',
    'bedroom_count',
    'fips_code'
    ]

plot_discrete(df, discrete_vars)

In [ ]:
# converting purchase date/transaction date to datetime type
# function::::

df['purchase_date'] = pd.to_datetime(df['purchase_date'])
df.info()

In [ ]:
# let's plot the purchase/transaction dates
# creating a new column/home purchase by 2017 month
# add function:::::

# creating a lambda function to isolate the dates by month and year
months_lst = df['purchase_date'].map(lambda dt: dt.strftime('%Y-%m'))

# adding the new 2017 month series to main df as new column
df["purchase_month"] = months_lst.astype("str")

grouped_df = df.sort_values("purchase_month").groupby('purchase_month').size().to_frame("count").reset_index()

sns.set(font_scale = .5, style = "darkgrid")
ax = sns.countplot(x = "purchase_month",
                data = df,
                order = grouped_df["purchase_month"],
                palette = "crest")

ax.bar_label(ax.containers[0])

plt.xlabel(None)
plt.title("2017 Home Purchases by Month")
plt.show()

In [ ]:
df.head()

----
**``establishing categorical/continuous variables/features``**

In [ ]:
df.columns.tolist()

In [ ]:
df.info()

In [ ]:
for col in df.columns:
    print(f'Columns name: {col} has {df[col].nunique()} number of unique values')
    print("---------------------")

In [ ]:
# converting "year_built" to "home_age"
# add function:::
import datetime

year_built = df.year_built.astype("int")
curr_year = datetime.datetime.now().year

# placing column/series back into main df
df["home_age"] = curr_year - year_built
df.head()

In [ ]:
unique_in_cols = {}

for col in df.columns:
    if col in unique_in_cols:
        unique_in_cols[col].append(df[col].nunique())
    else:
        unique_in_cols[col] = df[col].nunique()
    
unique_in_cols

In [ ]:
cat_lst = list()
cont_lst = list()

for key, value in unique_in_cols.items():
    if value <= 20:
        df[key] = df[key].astype("object")
        cat_lst.append(key)
    else:
        cont_lst.append(key)

print(cat_lst)
print(cont_lst)

-----
### ``Pre-processing:``

In [ ]:
# return back to this
# import geopandas as gpd
# import plotly.express as px
# import descartes

In [ ]:
df.columns.tolist()

In [ ]:
# creating a new column/feature to convert longitude and latitude to shape (location)
# df_geo = gpd.GeoDataFrame(df, geometry = gpd.points_from_xy(
#     df.longitude, df.latitude))

In [ ]:
# states = gpd.read_file("/Users/mijailmariano/codeup-data-science/geopandas-tutorial/data/usa-states-census-2014.shp")
# print(type(states))
# states.head()

In [ ]:
# get built-in dataset from geopandas
# df_geo.plot()

In [ ]:
# let's create a column for 1/2 bathrooms 

df["half_bathroom"] = df["bathroom_count"].astype("str").str.contains(".5").astype(bool)
df.columns.tolist()

In [ ]:
# renaming "purchase_month" values
# add function::::

year_and_month = df["purchase_month"].sort_values().unique().tolist()
month_lst = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September']

df["purchase_month"] = df["purchase_month"].replace(
    year_and_month,
    month_lst)

df.purchase_month.unique()

In [ ]:
# creating binned categorical columns for number of bathrooms

df["1_to_3.5_baths"] = df["bathroom_count"] <= 3.5
df["4_to_6.5_baths"] = (df["bathroom_count"] > 3.5) | (df["bathroom_count"] <= 6.5)

In [ ]:
# creating binned categorical columns for number of bedrooms

df["1_to_2_bedrooms"] = df["bedroom_count"] <= 2
df["3_to_4_bedrooms"] = (df["bedroom_count"] > 2) | (df["bedroom_count"] <= 4)
df["5_to_6_bedrooms"] = (df["bedroom_count"] > 4) | (df["bedroom_count"] <= 6)

In [ ]:
# let's also create a "season" column/feature where:
# winter months = jan., feb.
# spring months = mar., apr., may.
# summer months = june, jul., aug.
# fall = sep.

# df["winter_months"] = ((df["purchase_month"] == "January") | (df["purchase_month"] == "February")).astype(bool)
# df["spring_months"] = ((df["purchase_month"] == "March") | (df["purchase_month"] == "April") | (df["purchase_month"] == "May")).astype(bool)
# df["summer_months"] = ((df["purchase_month"] == "June") | (df["purchase_month"] == "July") | (df["purchase_month"] == "August")).astype(bool)
# df["fall_months"] = (df["purchase_month"] == "September").astype(bool)

In [ ]:
# add function:::

dummy_df = pd.get_dummies(df[[
    'fips_code', 
    'county_id',
    'purchase_month']])
    
dummy_df.head()

In [ ]:
# dropping redundant categorical cols from main df 
# add function::::

# dropping the following features/columns since they are being either 1. bucketed or 2. have created dummy variables
df = df.drop(columns = [
    'purchase_date',
    'purchase_month',
    'year_built', 
    'bedroom_count', 
    'bathroom_count', 
    'fips_code',
    'county_id'])

df = pd.concat([df, dummy_df], axis = 1)
df.shape

In [ ]:
# converting categorical columns/features to object/string type
df.info()

In [ ]:
df.columns.to_list()

In [ ]:
# 1st split: splitting the dataset in train, validate, and test

train, validate, test = train_validate_test_split(df)

In [ ]:
# establish baselines for the train and validation subsets
from math import sqrt

baseline = round(train["home_value"].mean(), 2)

train['baseline'] = baseline
validate['baseline'] = baseline

train_rmse = sqrt(mean_squared_error(train.home_value, train.baseline))
validate_rmse = sqrt(mean_squared_error(validate.home_value, validate.baseline))

print('Train baseline RMSE: {:.2f}'.format(train_rmse))
print('Validation baseline RMSE: {:.2f}'.format(validate_rmse))

In [ ]:
# dropping created baseline column before 2nd split
train = train.drop(columns = "baseline")
validate = validate.drop(columns = "baseline")

In [ ]:
# 2nd split: splitting larger datasets into x and y variables

X_train = train.drop(columns = "home_value")
y_train = train['home_value']

X_validate = validate.drop(columns = "home_value")
y_validate = validate['home_value']

X_test = test.drop(columns = "home_value")
y_test = test['home_value']

In [ ]:
print(X_train.shape)
print(y_train.shape)

In [ ]:
X_train.columns.tolist()

In [ ]:
X_train.info()

----
### ``Exploration:``

In [ ]:
# plotting correlation amongst all data features

plt.figure(figsize=(12, 5))
train.corr()["home_value"].sort_values(ascending = False).plot.barh(figsize=(12, 20))
plt.show()

----
#### **``Hypothesis Testing:``**

<u>Taget and Categorical/discrete Feature:</u> T-test


1. 'bathroom_count'
2. 'bedroom_count'
3. 'fips_code'
4. 'county_id'
5. 'purchase_month

<u>Taget and Continuous Feature:</u> Pearson R/correlation

1. 'living_sq_feet'
2. 'latitude'
3. 'longitude'
4. 'property_sq_feet'
5. 'city_id'
6. 'zip_code'
7. 'year_built'
8. 'home_age'

In [ ]:
X_train.select_dtypes(exclude = ["bool", "uint8"]).columns

In [ ]:
# sns visualizing all variables plotted against target  w/ line of best fit
# selecting initial variables to conduct a pair plot

pair_lst = X_train.select_dtypes(exclude = ["bool", "uint8"]).columns.tolist()

for col in pair_lst:
        plt.figure(figsize = (10, 4))
        sns.set(font_scale = 1)

        # plotting ea. feature against target variable with added alpha/transparency for aggregation areas
        ax = sns.regplot(train[col], \
        train["home_value"], \
        
        # adding superficial noise to independent variables to help visualize the individual plots
        scatter_kws = {'alpha': 1/10}, \
        line_kws={
            "color": "red", 'linewidth': 1.5})
        
        ax.figure.set_size_inches(18.5, 8.5)
        sns.despine()
        # removing scientific notations
        ax.ticklabel_format(style = "plain")
        
        # removing x_axis label
        ax.set_xlabel(None)

        plt.title(col)
        plt.show()

In [ ]:
train.info()

In [ ]:
cat_lst = train.select_dtypes(include = ["bool", "uint8"]).columns.tolist()
cont_lst = train.select_dtypes(exclude = ["object", "uint8", "bool"]).columns.tolist()

In [ ]:
# looping through categorical samples (dummy variables) to determine statistical relationship to target variable
import scipy.stats as stats 

target_mean = round(y_train.mean(), 2)
alpha = 0.05

metrics = []
for col in cat_lst:
    sub_group = train[train[col] == 1].home_value
    t_score, p_value = stats.ttest_1samp(sub_group, target_mean)

    if p_value < alpha:
        output = {
            "discrete feature": col,
            "t_score": t_score,
            "p_value": p_value}
        
        metrics.append(output)

    else:
        print(f'Column: {col} not statistically significant.')
        print("--------------------------------------")

categorical_scores = pd.DataFrame(metrics)
categorical_scores.round(4)

In [ ]:
# looping through continuous varibles to measure statistical realationship usign Pearson R method with target variable

target = train["home_value"]
alpha = 0.05

metrics = []

for col in cont_lst:
    r, p_value = stats.stats.pearsonr(train[col], target)

    if p_value < alpha:
        output = {
            "continuous feature": col,
            "correlation coeffficient": r,
            "p_value": p_value}
        
        metrics.append(output)

    else:
        print(f'Column: {col}')
        print(f'P_value: {round(p_value, 4)}')
        print('Not statistically significant.')
        print("--------------------------------------")

continuous_scores = pd.DataFrame(metrics)
continuous_scores.round(4)

----
**``Statiscal Analysis/Conclusion Notes:``**

Given the low/no statistical significance found in the following features or values, i will elect to drop or disregard the following feature options. Looking ahead, I will want to further examing/explore why these feature or options are considered "statistically insignificant" when determining "home_value".

<u>``Continuous features``</u>
1. "city_id"
2. "property_sq_feet"
3. "longitude"
4. "zip_code"

<u>``Categorical/discrete values:``</u>
1. '4_to_6.5_baths'
2. '3_to_4_bedrooms'
3. '5_to_6_bedrooms'
4. 'purchase_month_February'
5. 'purchase_month_June'

In [ ]:
# establishing new datasets for feature engineering and model pre-processing
# add function::::

X_train = X_train.drop(columns = [
    '4_to_6.5_baths',
    '3_to_4_bedrooms',
    '5_to_6_bedrooms',
    'purchase_month_February', 
    'purchase_month_June',
    "city_id", 
    "property_sq_feet",
    "longitude",
    "zip_code"])

X_validate = X_validate.drop(columns = [
    '4_to_6.5_baths',
    '3_to_4_bedrooms',
    '5_to_6_bedrooms',
    'purchase_month_February', 
    'purchase_month_June',
    "city_id", 
    "property_sq_feet",
    "longitude",
    "zip_code"])

# X_test = X_test.drop(columns = [
#     '4_to_6.5_baths',
#     '3_to_4_bedrooms',
#     '5_to_6_bedrooms',
#     'purchase_month_February', 
#     'purchase_month_June',
#     "city_id", 
#     "property_sq_feet",
#     "longitude",
#     "zip_code"])

In [ ]:
# model feature cleaning function
def clean_for_features(df):
    df = df.drop(columns = [
        '4_to_6.5_baths',
        '3_to_4_bedrooms',
        '5_to_6_bedrooms',
        'purchase_month_February', 
        'purchase_month_June',
        "city_id", 
        "property_sq_feet",
        "longitude",
        "zip_code"])

    return df

----
**``Feature engineering for MVP Variables/features:``**

which x variables do i believe will hold the most significance in modeling?

* purchase month
* living sq feet
* home age
* number of bathrooms/bedrooms


<u>``Scaling continuous training (independent) variables/features:``</u>

In [ ]:
# Robust Scaler Plots

cont_lst = X_train.select_dtypes(exclude = ["object", "uint8", "bool"]).columns.tolist()

for col in cont_lst:
    scaler = RobustScaler()
    scaler.fit(X_train[[col]])

    x_scaled = scaler.transform(X_train[[col]])

    plt.figure(figsize=(18, 6))
    plt.subplot(121)
    ax = sns.histplot(X_train[[col]], bins = 25, edgecolor = 'black', label = col)
    # removing axes scientific notation 
    ax.ticklabel_format(style = "plain") 
    plt.title(f'Original: {col}')
    plt.legend()

    plt.subplot(122)
    ax = sns.histplot(x_scaled, bins=25, edgecolor = 'black', label = "scaled")
    # removing axes scientific notation 
    ax.ticklabel_format(style = "plain") 
    plt.title(f'Scaled: {col}')
    plt.legend()

In [ ]:
# Standard Scaler

for col in cont_lst:
    scaler = StandardScaler()
    scaler.fit(X_train[[col]])

    x_scaled = scaler.transform(X_train[[col]])

    plt.figure(figsize=(18, 6))
    plt.subplot(121)
    ax = sns.histplot(X_train[[col]], bins = 25, edgecolor = 'black', label = col)
    # removing axes scientific notation 
    ax.ticklabel_format(style = "plain") 
    plt.title(f'Original: {col}')
    plt.legend()

    plt.subplot(122)
    ax = sns.histplot(x_scaled, bins=25, edgecolor = 'black', label = "scaled")
    # removing axes scientific notation 
    ax.ticklabel_format(style = "plain") 
    plt.title(f'Scaled: {col}')
    plt.legend()

In [ ]:
# MinMax Sccaler Plots

for col in cont_lst:

    scaler = MinMaxScaler()
    scaler.fit(X_train[[col]])

    x_scaled = scaler.transform(X_train[[col]])

    plt.figure(figsize=(18, 6))
    plt.subplot(121)
    ax = sns.histplot(X_train[[col]], bins = 25, edgecolor = 'black', label = col)
    # removing axes scientific notation 
    ax.ticklabel_format(style = "plain") 
    plt.title(f'Original: {col}')
    plt.legend()

    plt.subplot(122)
    ax = sns.histplot(x_scaled, bins=25, edgecolor = 'black', label = "scaled")
    # removing axes scientific notation 
    ax.ticklabel_format(style = "plain")
    plt.title(f'Scaled: {col}')
    plt.legend()

In [ ]:
# scaling continuous features/data with sklearn "MinMaxScaler"
# creating function to do this
# add function::::

def scaled_data(df, scaled_cols):
    # creating a copy of the original zillow/dataframe
    df_scaled = df.copy()

    scaler = RobustScaler()

    scaler.fit(df_scaled[scaled_cols])

    df_scaled[scaled_cols] = scaler.transform(df_scaled[scaled_cols])

    # returning newly created dataframe with scaled data
    return df_scaled

In [ ]:
# creaing a new list for scaling (excludes target variable)
scaled_lst = X_train.columns.tolist()
scaled_lst

In [ ]:
# generating new X_train, and X_validate datasets w/scaled data
X_train = scaled_data(X_train, scaled_lst)
X_validate = scaled_data(X_validate, scaled_lst)

In [ ]:
# passing kbest function on train dataset:

select_kbest(X_train, y_train, 10)

In [ ]:
# passing recursive function on train dataset:

recursive_feature_eng(X_train, y_train, 10).reset_index(drop=True)

In [ ]:
from sklearn.feature_selection import RFECV

# initiate, and fit
rfecv = RFECV(
    estimator = LinearRegression(),
    min_features_to_select = 5
)

rfecv = rfecv.fit(X_train, y_train)

In [ ]:
feature_lst = X_train.columns[rfecv.support_].tolist()
feature_lst

In [ ]:
lr = LinearRegression()
lr_model = lr.fit(X_train, y_train)

lars = LassoLars()
lars_model = lars.fit(X_train, y_train)

glm = TweedieRegressor(alpha = 1, power = 0)
glm_model = glm.fit(X_train, y_train)

print("Training R-squared:", lr_model.score(X_train, y_train))
print("Training R-squared:", lars_model.score(X_train, y_train))
print("Training R-squared:", glm_model.score(X_train, y_train))

In [ ]:
#Create a looping statement to evaluate all models
from math import sqrt

models = [lr_model, lars_model, glm_model]

for model in models:

    train_model = model.predict(X_train)
    rmse_train = sqrt(mean_squared_error(y_train,
                                         train_model))
    
    validate_model = model.predict(X_validate)
    rmse_validate = sqrt(mean_squared_error(y_validate,
                                         validate_model))
    
    print('RMSE for {} model on the train dataset: {}.'.format(model, round(rmse_train, 2)))
    print('RMSE for {} model on the validate dataset: {}.'.format(model, round(rmse_validate, 2)))
    print()

----
#### <u>**``Baseline Prediction & Evaluations:``**</u>

In [ ]:
# creating the independent and dependent variables
X_var = pd.DataFrame(X_validate[feature_lst])
y_var = pd.DataFrame(y_validate)

In [ ]:
model_df = pd.concat([X_var, y_var], axis = 1)
model_df.head()

In [ ]:
# creating a mean baseline prediction
baseline_mean_predictions = round(y_validate.mean(), 2)
baseline_mean_predictions

In [ ]:
# adding baseline to model_df
model_df["baseline_mean_predictions"] = baseline_mean_predictions
model_df.head()

In [ ]:
# generating validate model predictions and assigning to dataframe
lr_predictions = lr_model.predict(X_validate)
model_df["linear_predictions"] = lr_predictions.round(2)

lars_predictions = lars_model.predict(X_validate)
model_df["lars_predictions"] = lars_predictions.round(2)

glm_predictions = lars_model.predict(X_validate)
model_df["glm_predictions"] = glm_predictions.round(2)

In [ ]:
# checking the df
model_df.head()

In [ ]:
def regression_errors(y, y_hat):
    # importing math.sqrt module for calculations
    from math import sqrt
    
    # generating model residuals and residuals squared
    df = y - y_hat
    df["residual^2"] = df.round(2) ** 2

    # generating sum of squared error
    SSE = sum(df["residual^2"])

    # generating explained sum of squares
    ESS = sum((y_hat - y.mean()) ** 2)

    # generating total sum of squares error
    TSS = ESS + SSE

    # generating mean squared error
    MSE = SSE/len(y)

    # generating root mean squared error
    RMSE = sqrt(MSE)

    return SSE, ESS, TSS, MSE, RMSE

In [ ]:
# getting error reports for baseline predictions

SSE, ESS, TSS, MSE, RMSE = regression_errors(model_df["home_value"], model_df["baseline_mean_predictions"])

print('Baseline SSE = {:.1f}'.format(SSE))
print('Baseline ESS = {:.1f}'.format(ESS))
print('Baseline TSS = {:.1f}'.format(TSS))
print('Baseline MSE = {:.1f}'.format(MSE))
print('Baseline RMSE = {:.1f}'.format(RMSE))

In [ ]:
# getting error reports for linear model predictions

SSE, ESS, TSS, MSE, RMSE = regression_errors(model_df["home_value"], model_df["linear_predictions"])

print('Model SSE = {:.1f}'.format(SSE))
print('Model ESS = {:.1f}'.format(ESS))
print('Model TSS = {:.1f}'.format(TSS))
print('Model MSE = {:.1f}'.format(MSE))
print('Model RMSE = {:.1f}'.format(RMSE))

In [ ]:
def get_error_report(y, y_hat):
    # importing math.sqrt module for calculations
    from math import sqrt
    
    # generating model residuals and residuals squared
    df = y - y_hat
    df["residual^2"] = df.round(2) ** 2

    # generating sum of squared error
    SSE = sum(df["residual^2"])

    # generating explained sum of squares
    ESS = sum((y_hat - y.mean()) ** 2)

    # generating total sum of squares error
    TSS = ESS + SSE

    # generating mean squared error
    MSE = SSE/len(y)

    # generating root mean squared error
    RMSE = sqrt(MSE)

    print(f'{y_hat.name} SSE: {SSE}')
    print(f'{y_hat.name} ESS: {ESS}')
    print(f'{y_hat.name} TSS: {TSS}')
    print(f'{y_hat.name} MSE: {MSE}')
    print(f'{y_hat.name} RMSE: {RMSE}')

    return SSE, ESS, TSS, MSE, RMSE

In [ ]:
SSE, ESS, TSS, MSE, RMSE = get_error_report(model_df["home_value"], model_df["glm_predictions"])

In [ ]:
# final error report df

final_report = {
'Models': ['Baseline Predictions', 'Linear Regression Predictions', 'Tweedie Regressor Predictions'],
'Explained Sum of Squares (ESS)': [0.3, 121233107350265.9, 114322513495073.8],
'Mean Sum Error (MSE)': [41914677866.4, 32010289438.0, 32057696544.18],
'Root Mean Squared Error (RMSE)': [204730.7, 178914.2, 179046.63]
}

pd.set_option('display.float_format', lambda x: '%.2f' % x)
final_report = pd.DataFrame(final_report)
final_report.head()

In [ ]:
# plotting actual home values, baseline_mean_predictions predictions, and model predictions
# add function:::

plt.figure(figsize = (16, 10))

# plotting baseline prediction line of best fit
plt.plot(model_df['home_value'].sample(300, 
        random_state = 123), 
        model_df['baseline_mean_predictions'].sample(300, random_state = 123), 
        alpha=0.5,
        color='red', 
        ls = ':', 
        label='_nolegend_')

# plotting home value line of best fit
plt.plot(model_df['home_value'].sample(300, random_state = 123), 
        model_df['home_value'].sample(300, random_state = 123), 
        alpha=0.5,
        color='purple', 
        label='_nolegend_')

# linear model plot
plt.scatter(model_df['home_value'].sample(300, random_state = 123), 
            model_df['linear_predictions'].sample(300, random_state = 123), 
            alpha=0.5,
            color='red', 
            s=100,
            label='Linear Regression')

# lasso lars plot
plt.scatter(model_df['home_value'].sample(300, random_state = 123), 
            model_df['lars_predictions'].sample(300, random_state = 123), 
            alpha=0.5,
            color='yellow', 
            s=100, 
            label='Lasso Lars')

# tweedie/glm plot
plt.scatter(model_df['home_value'].sample(300, random_state = 123), 
            model_df['glm_predictions'].sample(300, random_state = 123), 
            alpha=0.5,
            color='green', 
            s=100, 
            label='Tweedie Regressor')


plt.legend()
plt.xlabel("Actual Home Value")
plt.ylabel('Predicted Home Value')
plt.title('Actual Home Values vs Model Predicted Home Values')

plt.show()

In [ ]:
# Model Residual (error) Plot

plt.figure(figsize=(16,8))
plt.axhline(label='No Error', 
            color = 'purple',
            ls = ':')

# plotting linear model
plt.scatter(model_df['home_value'].sample(300, random_state = 123), 
            model_df['linear_predictions'].sample(300, random_state = 123) - model_df['home_value'].sample(300, random_state = 123), 
            alpha=0.5,
            color='red', 
            s=100, 
            label='Linear Regression')

# plotting lasso lars model
plt.scatter(model_df['home_value'].sample(300, random_state = 123), 
            model_df['lars_predictions'].sample(300, random_state = 123) - model_df['home_value'].sample(300, random_state = 123), 
            alpha=0.5,
            color='yellow', 
            s=100, 
            label='Lasso Lars')

# plotting tweedie model
plt.scatter(model_df['home_value'].sample(300, random_state = 123), 
            model_df['glm_predictions'].sample(300, random_state = 123) - model_df['home_value'].sample(300, random_state = 123), 
            alpha=0.5,
            color='green', 
            s=100, 
            label='Tweedie Regressor')

plt.legend()
plt.xlabel('Actual Home Value')
plt.ylabel('Residual Error')
plt.title('Model Residual Plot')

plt.show()

In [ ]:
# Distribution of my model predictions (linear & tweedie)
# add function:::

plt.figure(figsize = (16, 8))
sns.set(style = "darkgrid")

plt.hist(model_df['home_value'], color='blue', alpha=0.4, label='Actual Home Value')
plt.hist(model_df['linear_predictions'], color='red', alpha=0.4, label='Linear Regression')
plt.hist(model_df['lars_predictions'], color='green', alpha=0.4, label='Laso Lars')


plt.xlabel('Home Value')
plt.ylabel('Frequency')
plt.title('Frequency of Home Values by Predictive Model')
plt.legend()

plt.show()

In [ ]:
# final model predictions and score on Test Dataset

test_df = pd.DataFrame(y_test)
print(test_df.shape )
test_df.head() # checks out!

In [ ]:
# scaling X_test data using created scaling function

X_test = scaled_data(X_test, scaled_lst)

In [ ]:
# cleaning X_test dataset for model features

X_test = clean_for_features(X_test)
X_test.shape # checks out!

In [ ]:
test_df["model_predictions"] = lr_model.predict(X_test)
test_df.head()

In [ ]:

rmse_test = sqrt(mean_squared_error(test_df['home_value'], test_df['model_predictions']))
print(f'RMSE for OLS model on the test dataset: {round(rmse_test, 2)}')

In [ ]:
final_rmse = pd.DataFrame({
    "Test": ["Baseline", "Train", "Validate", "Final"],
    "RMSE": [204730.70,178296.33,178914.20,177111.14],
    "Relative Diff.": [0, .15, 0, .01]})

In [ ]:
final_rmse